In [1]:
from astropy.io import fits
import astropy.units as u
from astroquery import ned
from astropy.wcs import wcs
from astropy.coordinates import SkyCoord
from astropy.table import Table, vstack, hstack
import numpy as np
import tables
from tabulate import tabulate
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from math import ceil

ModuleNotFoundError: No module named 'astropy'

In [2]:
from tqdm import tqdm

In [3]:
%matplotlib inline
import mpld3
mpld3.enable_notebook()

In [4]:
# This is a cell with all self-defined functions
def coord_NED(dwarfname):
    from astroquery.ned.core import RemoteServiceError
    # get rid of the weird symbos *, (, ), etc, since NED doens't like it
    newname = dwarfname.replace('*','').replace('(','').replace(')', '')

    try:
        target = ned.Ned.query_object(newname)
        ned_exist = True
    except RemoteServiceError: 
        print('NED does not have info, skip: %s/%s\n'%(dwarfname, newname))
        ned_exist= False 
    
    if ned_exist == False:
        tRA, tDEC, tVel, tDist = np.nan, np.nan, np.nan, np.nan
    else: # if NED does have this target, then extract coordinates
        tRA = target["RA(deg)"].data.data[0]
        tDEC = target["DEC(deg)"].data.data[0]
        tVel = target["Velocity"].data.data[0]
        tDist = target["Distance (arcmin)"].data.data[0]
    return tRA, tDEC, tDist, tVel, ned_exist

## transform from vhelio to vlsr 
def vhelio2vlsr_Westmeier(vel_init, ral, decb, reverse=False, doradec=True):
    '''
    - from http://www.atnf.csiro.au/people/Tobias.Westmeier/tools_hihelpers.php
    - obj_ra:  should be in degree
    - obj_dec: should be in degree
    - vel_init: velocity that need to be transformed
    -          vel_init = vhelio if reverse = False
    -          vel_init = vlsr   if reverse = True
    - Favor this one than the other one from Rosolowsky
    '''
    from astropy.coordinates import SkyCoord
    import numpy as np

    if doradec==True:
        c = SkyCoord(ral, decb, unit='deg')
        l = np.radians(c.galactic.l.value)
        b = np.radians(c.galactic.b.value)
    else:
        l = np.radians(ral)
        b = np.radians(decb)
    # vlsr 00> vhelio
    if reverse:
        delv = -(9*np.cos(l)*np.cos(b)+12*np.sin(l)*np.cos(b)+7*np.sin(b))
    else:
        delv = +9*np.cos(l)*np.cos(b)+12*np.sin(l)*np.cos(b)+7*np.sin(b)

    # print 'Velocity correction at this (RA, DEC) is (km/s): ', delv
    return vel_init+delv


def find_the_cube(ra, dec, observation='HI4PI'):
    '''
    Use the input (ra, dec) to decide which cube the data locates.

    observation: most of the time, it is HI4PI or GALFA-HI . 
    
    Note: GALFA-HI table is not available now, will do later 
    '''

    clt = Table.read('tables/%s_RADEC.dat'%(observation), format='ascii')
    cramin, cramax = clt['min_ra'], clt['max_ra']
    cdcmin, cdcmax = clt['min_dec'], clt['max_dec']
    indra = np.all([ra>cramin, ra<cramax], axis=0)
    inddc = np.all([dec>cdcmin, dec<cdcmax], axis=0)
    indall = np.where(np.all([indra, inddc], axis=0) == True)

    cubename = clt['cubename'][indall]
    if len(cubename)==0:
        return ''
    else:
        cubename = cubename[0]

        return cubename
    
import sys
import astropy.wcs as wcs
import numpy as np


# a function to read in header and generate RA, DEC and VLSR array
def get_cubeinfo(header, returnHeader=False):
    '''
    A function created to parse the RA, DEC, (and velocity) information from the 2D (3D) header

    - This function has been tested with GALFA-HI/EBHIS cubes, and GALFA-HI 2D images.
    -               also been tested with LAB cubes/images that are in (glon, glat) coordinates.
    - The input header can be 2D: NAXIS=2. NAXIS1 is RA (glon), 2 is DEC (glat)
    -                      or 3D: NAXIS=3. NAXIS1 is RA (glon), 2 is DEC (glat), 3 is Velocity
    - Return: if GALFA-HI or EBHIS:
                        ra and dec in 2D, with shape of (dec.size, ra.size) or (NAXIS2, NAXIS1)
    -                   velocity in 1D.
                        or (ra, dec, vlsr, header array)
    -         if LAB:
                        glon, glat in 2D, with shape of (glat.size, glon.size) or (NAXIS2, NAXIS1)
    -                   velocity in 1D.
                        or (gl, gb, vlsr, header array)
    - History: updated as of 2016.10.03. Yong Zheng @ Columbia Astro.
    '''


    hdrarrs = []
    if header['NAXIS'] == 2:
        hdr2d = header.copy()
        hdrarrs.append(hdr2d)
    elif header['NAXIS'] == 3:
        # create a 2D header (RA/DEC) to speed up the RA/DEC calculation using astropy.wcs
        hdr2d = header.copy()
        # we don't need the velocity (3) information in the header
        delkey = []
        for key in hdr2d.keys():
            if len(key) != 0 and key[-1] == '3': delkey.append(key)
        for i in delkey: del hdr2d[i]

        hdr2d['NAXIS'] = 2
        if 'WCSAXES' in hdr2d.keys(): hdr2d['WCSAXES']=2
        # create a 1D header (vel) to parse the velocity using astropy.wcs
        hdr1d = header.copy()
        # we don't need the RA/DEC keywords info in the header now.
        delkey = []
        for keya in hdr1d.keys():
            if len(keya) != 0 and keya[-1] in ['1', '2']: delkey.append(keya)
        for i in delkey: del hdr1d[i]
        delkey = []
        for keyb in hdr1d.keys():
            if len(keyb) != 0 and keyb[-1] == '3':
                hdr1d.append('%s1'%(keyb[:-1]))
                hdr1d['%s1'%(keyb[:-1])] = hdr1d[keyb]
                delkey.append(keyb)
        for i in delkey: del hdr1d[i]
        hdr1d['NAXIS'] = 1
        if 'WCSAXES' in hdr1d.keys(): hdr1d['WCSAXES']=1

        # save header arrays
        hdrarrs.append(hdr2d)
        hdrarrs.append(hdr1d)
    else:
        print("This code can only handle 2D or 3D data")
        sys.exit(1)

    return_arrays = []

    # calculate RA, DEC
    gwcsa = wcs.WCS(hdr2d)
    n1, n2 = hdr2d['NAXIS1'], hdr2d['NAXIS2']
    ax = np.reshape(np.mgrid[0:n1:1]+1, (1, n1))  # For FITS standard, origin = 1
    ay = np.reshape(np.mgrid[0:n2:1]+1, (n2, 1))  #   then for numpy standard, origin = 0
    coor1, coor2 = gwcsa.all_pix2world(ax, ay, 1) # coor1 = ra  or glon
    return_arrays.append(coor1)                   # coor2 = dec or glat
    return_arrays.append(coor2)

    ## calculate VLSR
    if header['NAXIS'] == 3:
        gwcsb = wcs.WCS(hdr1d)
        n1 = hdr1d['NAXIS1']
        ax = np.mgrid[0:n1:1]+1
        # ax = np.linspace(0, n1, n1)  # nope, wrong
        vel = gwcsb.all_pix2world(ax, 1)[0]
        if 'CUNIT1' in hdr1d.keys():
            if hdr1d['CUNIT1'] in ['m/s', 'M/S', 'M/s', 'm/S']:
                vel = vel/1e3
        else: vel = vel/1e3  # default is usually in m/s
        return_arrays.append(vel)

    if returnHeader == True: return_arrays.append(hdrarrs)
    return return_arrays

In [5]:
def ellipse_mask_cube(semi_a, semi_b, PA, tar_x, tar_y, cube_header, extend_pix = 0):
    ## make ellipse based on the semi major/minor axies and position angle of the targets 
    
    cube_wh = cube_header['NAXIS1']  # RA
    cube_ht = cube_header['NAXIS2']  # DEC
    x1d = np.arange(cube_wh)
    y1d = np.arange(cube_ht)
    x2d, y2d = np.meshgrid(x1d, y1d)

    # center at the source 
    x2d_nc, y2d_nc = x2d-tar_x, y2d-tar_y

    # this is a rough pixel length for the semimajor and semiminor axis
    semi_a_pix = ceil(abs(semi_a / cube_header['CDELT1']))
    semi_b_pix = ceil(abs(semi_b / cube_header['CDELT1']))
    
    #phi = np.radians(90-PA) changing 
    phi = np.radians(90+PA)

    # transform to new coordinate system
    xx_prime = x2d_nc*np.cos(phi) + y2d_nc*np.sin(phi)
    yy_prime = -x2d_nc*np.sin(phi) + y2d_nc*np.cos(phi)

    mask_ep = ((xx_prime/(semi_a_pix+extend_pix))**2 + (yy_prime/(semi_b_pix+extend_pix))**2)<=1 
    return mask_ep

In [6]:
def find_info(objname):
    from astropy.table import Table 
    tb = Table.read('/Users/amalyajohnson/Desktop/astro/Dwarf Galaxies/dwarfgalcomplete.txt', format='ascii')
    
    newname = objname.replace('*','').replace('(','').replace(')', '').replace(' ', '')
    for i, iname in enumerate(tb['GalaxyName']):
        i_tbnewname = iname.replace('*','').replace('(','').replace(')', '').replace(' ', '')
        
        if newname == i_tbnewname:
            find_it = True
            break
        else: 
            find_it = False
            
    if find_it == True:
        print('YES, we find it!!!', tb['RA'][i], tb['DEC'][i], tb['vh(m/s)'][i])
        return tb['RA'][i], tb['DEC'][i], tb['vh(m/s)'][i], tb['rh(arcmins)'][i], tb['e=1-b/a'][i], tb['PA'][i]
    else:
        print('Sorry, not there, check your object name. ')
        return np.nan, np.nan, np.nan
    

# Galaxy Info

In [202]:
objname = 'Pegasus3' 

objRA, objDEC, objV_helio, a, ecc, PA = find_info(objname)
vcoord = vhelio2vlsr_Westmeier(0, objRA, objDEC, doradec=True) #in km/s
objV = objV_helio + vcoord*1e3
semi_b = (1 - ecc)*a/60 #degree, semi minor axis
semi_a = a/60 #degree, semi major axis

#For unresolved undetected only looking at 30 arcmin region:  
semi_a = .25  
semi_b = .25

YES, we find it!!! 336.092 5.42 999900


In [203]:
cubename = '/Users/amalyajohnson/Desktop/datacubes/DR1 GALFA/GALFA_HI_RA+DEC_332.00+02.35_W' #pull correct cube from directory
img1 = fits.getdata(cubename)
hdr = fits.getheader(cubename)

In [204]:
# setting up the WCS
wcsleop =  wcs.WCS(hdr)

# converting to pixel space
tar_x, tar_y, tar_v = wcsleop.all_world2pix(objRA, objDEC, objV, 0)

tar_x = int(tar_x)
tar_y = int(tar_y)
tar_v = int(tar_v)

# & checking it's in there, make sure the shape fits the size of tar_v, tar_x, tar_y
img1.shape,  tar_v, tar_x, tar_y
#help(wcsleop.all_world2pix)

### get the whole RA/DEC/Vel channel from the header 
all_ra, all_dec, all_vlsr = get_cubeinfo(hdr)  # all_vlsr is in km/s

In [205]:
# figure out how large the area surrounding the source that we want 
ep1 = ellipse_mask_cube(semi_a, semi_b, PA, tar_x, tar_y, hdr, extend_pix=0) #30 arcmin diameter because already changed semi_a and semi_b initially
ep2 = ellipse_mask_cube(.5, .5, PA, tar_x, tar_y, hdr, extend_pix = 0) #1 degree diameter

# No Velocity:

In [206]:
#all_vlsr[tar_v] # the velocity where the target central velocity channel is 

In [207]:
def std_array(objname, datacube): 
    
    vel_bins = np.mgrid[-750:750:10]  # (-600, -590, .... 590, 600) bin size in velocity km/s
    #-600 to 601 for hi4pi, -750 to 751 for GALFA-HI
    flux_stds = np.zeros(vel_bins.size)
    
    for i, iv in enumerate(vel_bins):
        # find the closest velocity to iv 
        v_chan = np.argmin(np.fabs(all_vlsr-iv))
        #print(v_chan, iv, all_vlsr[v_chan])      # and test iv and all_vlsr[v_chan] are indeed the closest   
        f_std = np.nanstd(datacube[v_chan][ep1]) # ep1 is 30 arcmin beam, ep2 would be 1 degree
        flux_stds[i] = f_std
    return vel_bins, flux_stds

vel_bins, flux_stds = std_array(objname, img1)
# vel_bins, np.arange(all_vlsr.min(), all_vlsr.max(), 10)

std_array(objname, img1)

vel_bins

array([-750, -740, -730, -720, -710, -700, -690, -680, -670, -660, -650,
       -640, -630, -620, -610, -600, -590, -580, -570, -560, -550, -540,
       -530, -520, -510, -500, -490, -480, -470, -460, -450, -440, -430,
       -420, -410, -400, -390, -380, -370, -360, -350, -340, -330, -320,
       -310, -300, -290, -280, -270, -260, -250, -240, -230, -220, -210,
       -200, -190, -180, -170, -160, -150, -140, -130, -120, -110, -100,
        -90,  -80,  -70,  -60,  -50,  -40,  -30,  -20,  -10,    0,   10,
         20,   30,   40,   50,   60,   70,   80,   90,  100,  110,  120,
        130,  140,  150,  160,  170,  180,  190,  200,  210,  220,  230,
        240,  250,  260,  270,  280,  290,  300,  310,  320,  330,  340,
        350,  360,  370,  380,  390,  400,  410,  420,  430,  440,  450,
        460,  470,  480,  490,  500,  510,  520,  530,  540,  550,  560,
        570,  580,  590,  600,  610,  620,  630,  640,  650,  660,  670,
        680,  690,  700,  710,  720,  730,  740])

In [208]:
rowv = flux_stds

np.nanmedian(flux_stds) #= med? 

0.052954327315092087

In [209]:
#median around: looking at by eye
med = .05
row_med = ([objname, med])

In [211]:
tbvel = Table(names = vel_bins)
tbvel.add_row(rowv)

tbmed = Table(names=('GalaxyName', 'med'), 
             meta={'name': 'first table'},
              dtype=('<U11', 'f8'))

tbmed.add_row(row_med)

tbvel
tb = hstack([tbmed, tbvel])

f = open('galfa_novelocity.txt', 'w')
table = tb
f.write(tabulate(tb))
f.close

tb

GalaxyName,med,-750,-740,-730,-720,-710,-700,-690,-680,-670,-660,-650,-640,-630,-620,-610,-600,-590,-580,-570,-560,-550,-540,-530,-520,-510,-500,-490,-480,-470,-460,-450,-440,-430,-420,-410,-400,-390,-380,-370,-360,-350,-340,-330,-320,-310,-300,-290,-280,-270,-260,-250,-240,-230,-220,-210,-200,-190,-180,-170,-160,-150,-140,-130,-120,-110,-100,-90,-80,-70,-60,-50,-40,-30,-20,-10,0,10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200,210,220,230,240,250,260,270,280,290,300,310,320,330,340,350,360,370,380,390,400,410,420,430,440,450,460,470,480,490,500,510,520,530,540,550,560,570,580,590,600,610,620,630,640,650,660,670,680,690,700,710,720,730,740
str11,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
Pegasus3,0.05,1.52587890625e-05,1.52587890625e-05,1.52587890625e-05,1.52587890625e-05,12.1518945694,0.0702108591795,0.0618136264384,0.0517987422645,0.0530429035425,0.0520472154021,0.04904390499,0.0577242597938,0.0649710968137,0.064019292593,0.0583398379385,0.0616272725165,0.0610166043043,0.0606604106724,0.060619469732,0.0600510910153,0.0590834245086,0.0600789375603,0.0539867915213,0.0593913570046,0.057037435472,0.0512610338628,0.0493445284665,0.0529550015926,0.0580542869866,0.0548736155033,0.0566315092146,0.0531540848315,0.0661942884326,0.0604238584638,0.0613946355879,0.0630280524492,0.0565187968314,0.0604294873774,0.0542073771358,0.0590638257563,0.0609341450036,0.0594279244542,0.0529536530375,0.0498120374978,0.0522583387792,0.0620240010321,0.0644536912441,0.0616792216897,0.0544615350664,0.0519843399525,0.0590311288834,0.0552024543285,0.0588218271732,0.0589235313237,0.0531166754663,0.0584451630712,0.0530618391931,0.0570667050779,0.0606537684798,0.0574893914163,0.0611517801881,0.0546850822866,0.0544044151902,0.057630803436,0.0580237284303,0.0535509884357,0.0504878796637,0.0584247373044,0.0617797970772,0.0594453588128,0.056074988097,0.0536369495094,0.0529033951461,0.0797229409218,1.81794095039,1.67410850525,0.725980818272,0.0747250244021,0.0593903623521,0.0552567727864,0.043791256845,0.0423352867365,0.0531145222485,0.0481563694775,0.0434575229883,0.0395965203643,0.0398367568851,0.0431990884244,0.0456782393157,0.0460207499564,0.0502278134227,0.0428024716675,0.0462703071535,0.0477574802935,0.0477234274149,0.0497674532235,0.0413969382644,0.0468579754233,0.0458323284984,0.0433240644634,0.0478004254401,0.0489499643445,0.0441944636405,0.0474888905883,0.0506392195821,0.0553850494325,0.047641415149,0.0423948802054,0.0510342828929,0.0400293543935,0.0444069653749,0.0458653867245,0.0496945083141,0.0432445183396,0.0451038740575,0.0418308824301,0.0469280257821,0.0482001639903,0.0479100979865,0.0453667007387,0.0457739681005,0.0459159128368,0.0535564906895,0.0473573356867,0.053309969604,0.0464741401374,0.043821234256,0.0536215007305,0.0433210246265,0.0473789833486,0.0433113612235,0.0433566719294,

# Table

In [ ]:
tba = Table(names=('GalaxyName', 'med_std'), 
             meta={'name': 'first table'},
              dtype=('<U11', 'f8'))
tbnew.add_row(rowa)


#tb = Table.read('/Users/amalyajohnson/Desktop/astro/Dwarf Galaxies/hi4pi_resolved_undetected_table.txt', format='ascii')
#tb.add_row(rowd)

tb = (hstack([tba, tbb]))

f = open('galfa_unresolved_undetected_range.txt', 'w')
table = tb
f.write(tabulate(tb))
f.close

tb


# Std30_med, If Velocity Defined: 

In [ ]:
#a velocity channels, optical size of galaxy, this has been extended to 30arcmin for them all from the beginning 
a_1 = (img1[tar_v-5][ep1])
a_2 = (img1[tar_v-4][ep1])
a_3 = (img1[tar_v-3][ep1])
a_4 = (img1[tar_v-2][ep1])
a_5 = (img1[tar_v-1][ep1])
a_6 = (img1[tar_v][ep1])
a_7 = (img1[tar_v+1][ep1])
a_8 = (img1[tar_v+2][ep1])
a_9 = (img1[tar_v+3][ep1])
a_10 = (img1[tar_v+4][ep1])
a_11 = (img1[tar_v+5][ep1])
arange = np.array([a_1, a_2, a_3, a_4, a_5, a_6, a_7, a_8, a_9, a_10, a_11])
arange_std = np.array([np.nanstd(a_1), np.nanstd(a_2), np.nanstd(a_3), np.nanstd(a_4),
                       np.nanstd(a_5), np.nanstd(a_6), np.nanstd(a_7), np.nanstd(a_8),
                       np.nanstd(a_9), np.nanstd(a_10), np.nanstd(a_11)])


#b velocity channels, 1 degree diameter    
b_1 = (img1[tar_v-5][ep2])
b_2 = (img1[tar_v-4][ep2])
b_3 = (img1[tar_v-3][ep2])
b_4 = (img1[tar_v-2][ep2])
b_5 = (img1[tar_v-1][ep2])
b_6 = (img1[tar_v][ep2])
b_7 = (img1[tar_v+1][ep2])
b_8 = (img1[tar_v+2][ep2])
b_9 = (img1[tar_v+3][ep2])
b_10 = (img1[tar_v+4][ep2])
b_11 = (img1[tar_v+5][ep2])
brange = np.array([b_1, b_2, b_3, b_4, b_5, b_6, b_7, b_8, b_9, b_10, b_11])
brange_std = np.array([np.nanstd(b_1), np.nanstd(b_2), np.nanstd(b_3), np.nanstd(b_4), 
                       np.nanstd(b_5), np.nanstd(b_6), np.nanstd(b_7), np.nanstd(b_8), 
                       np.nanstd(b_9), np.nanstd(b_10), np.nanstd(b_11)])

med_stda = np.nanmedian(arange_std)
med_stdb = np.nanmedian(brange_std)
arange_std, med_stda 

#arange_std: stdev of 30arcmin region at the location of the galaxy through +/- 5 velocity channels from the central velocity
#med_stda: the median of this list of standard deviations
rowa = objname, '%.7f'%(med_stda)
rowb = arange_std
med_stda

# Table

In [ ]:
#tba = Table(names=('GalaxyName', 'med_std'), 
#             meta={'name': 'first table'},
#              dtype=('<U11', 'f8'))
tba.add_row(rowa)

#tbb = Table(names=('v_-5', 'v_-4', 'v_-3', 'v_-2', 
#                      'v_-1', 'v_0', 'v_+1', 'v_+2', 'v_+3', 'v_+4', 'v_+5'), 
#             meta={'name': 'first table'},
#              dtype=('f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8'))
tbb.add_row(rowb)

#tb = Table.read('/Users/amalyajohnson/Desktop/astro/Dwarf Galaxies/hi4pi_resolved_undetected_table.txt', format='ascii')
#tb.add_row(rowd)

tb = (hstack([tba, tbb]))

f = open('galfa_unresolved_undetected_range.txt', 'w')
table = tb
f.write(tabulate(tb))
f.close

tb


# MHI

In [ ]:
def dist(objname):
    tbm = Table.read('/Users/amalyajohnson/Desktop/astro/Dwarf Galaxies/dwarfgalcomplete_newnew.txt', format='ascii')
    for i, iname in enumerate(tbm['GalaxyName']):
        if iname == objname:
            find_it = True
            break 
        else: 
            find_it = False
        
    if find_it == True:
        dist = float(tbm['dist(kpc)'][i])
        return dist 

print(dist(objname), objname)

In [ ]:
# MHI = (2.36x10**5)(D**2 mpc)(Stot) for unresolved
x = 2.36*1e5
dmpc = dist(objname)/1e3  #mpc 
tflux = 0.030336279 #put in by hand because every table different/not exactly formatted
stot = tflux*10 #tflux always multiplied by 10km/s

#galfa: 9.1 k/Jy hi4pi: .6 jy/k
g = 1/9.1 
h = .6


MHI = x*(dmpc**2)*(stot*g) #multiply by 'h' or 'g' (hi4pi or galfa) to convert to janskys 
y = MHI
'%.5f'%(y)  #show 5 decimal points

In [ ]:
x = plt.scatter(vel_bins, flux_stds)
plt.show(x)

# Plot:

In [ ]:
plt.figure(figsize=(8, 8))
im = plt.imshow(img1[tar_v, :, :], cmap='jet', origin='lower' , vmin=-.5, vmax=.5)
plt.contour(ep1, colors='white', label='Optical Size')
plt.contour(ep2, colors='red', label='30 Arcmin region')
plt.text(tar_x+15, tar_y+15, objname, color='w', fontsize=14, weight='semibold')
plt.title(objname)
plt.colorbar(im)

#plt.xlim(0, 50)
#plt.ylim(75, 125)

#plt.savefig('/Users/amalyajohnson/Desktop/unresolved undetected/%s_image1_hi4pi.pdf'%(objname)) #to save file
plt.show()